In [2]:
from dotenv import load_dotenv
load_dotenv()

True

In [5]:
from langchain_openai import ChatOpenAI
llm = ChatOpenAI(model="gpt-4o-mini")

from langchain_core.output_parsers import StrOutputParser
parser = StrOutputParser()

chain = llm | parser


In [6]:
from langchain_core.chat_history import BaseChatMessageHistory, InMemoryChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory

store = {} # 특정 대화를 기록하기 위해 사용

def get_session_history(sessionId: str) -> BaseChatMessageHistory:
    if sessionId not in store: 
        store[sessionId] = InMemoryChatMessageHistory()
    return store[sessionId]

with_message_history = RunnableWithMessageHistory(chain, get_session_history)
config = {
    "configurable": {
        "session_id": "chat"
    }
}


In [7]:
from langchain_core.messages import HumanMessage, SystemMessage, AIMessage

messages = [
    SystemMessage(content="사용자의 질문에 2문장 이내로 짧게 대답해"),
    HumanMessage(content="오늘은 피자를 먹어야지!"),
    AIMessage(content="정말 좋은 생각이야. 음료는 무엇으로 할 거야?"),
    HumanMessage(content="내일은 수영을 가야지!"),
    AIMessage(content="수영이라니, 정말 좋은 운동이야. 수영장은 어디로 다녀?"),
    HumanMessage(content="주말에는 영화를 보러 갈 거야!"),
    AIMessage(content="주말이 벌써부터 기다려지겠는걸? 보려고 생각해둔 영화가 있어?"),
]

In [8]:
with_message_history.invoke(messages + 
  [HumanMessage(content="주말에 내가 뭐하러 간다고 했지?")],
  config=config
)


'주말에 영화를 보러 간다고 했어! 어떤 영화 볼 계획이야?'

In [9]:
with_message_history.invoke(
    [HumanMessage(content="방금 내가 무슨 질문을 했지?")],
    config=config
)

'주말에 네가 무엇을 하러 간다고 했는지 물어봤어.'

In [12]:
with_message_history.invoke(
    [HumanMessage(content="수영은 언제가기로 했지?")],
    config=config
)

[chain/start] [chain:RunnableWithMessageHistory] Entering Chain run with input:
[inputs]
[chain/start] [chain:RunnableWithMessageHistory > chain:load_history] Entering Chain run with input:
[inputs]
[chain/end] [chain:RunnableWithMessageHistory > chain:load_history] s] Exiting Chain run with output:
[outputs]
[chain/start] [chain:RunnableWithMessageHistory > chain:check_sync_or_async] Entering Chain run with input:
[inputs]
[chain/start] [chain:RunnableWithMessageHistory > chain:check_sync_or_async > chain:RunnableSequence] Entering Chain run with input:
[inputs]
[llm/start] [chain:RunnableWithMessageHistory > chain:check_sync_or_async > chain:RunnableSequence > llm:ChatOpenAI] Entering LLM run with input:
{
  "prompts": [
    "System: 사용자의 질문에 2문장 이내로 짧게 대답해\nHuman: 오늘은 피자를 먹어야지!\nAI: 정말 좋은 생각이야. 음료는 무엇으로 할 거야?\nHuman: 내일은 수영을 가야지!\nAI: 수영이라니, 정말 좋은 운동이야. 수영장은 어디로 다녀?\nHuman: 주말에는 영화를 보러 갈 거야!\nAI: 주말이 벌써부터 기다려지겠는걸? 보려고 생각해둔 영화가 있어?\nHuman: 주말에 내가 뭐하러 간다고 했지?\nAI: 주말에 영화를 보러 간다고 했어! 어

'내일 수영을 가기로 했다고 했어.'

In [11]:
import langchain
langchain.debug = True